# Quickstart with BentoML and XGBoost

Link to source code: https://github.com/bentoml/BentoML/tree/main/examples/xgboost

In [ ]:
import xgboost
import bentoml

### Train a classifier using the agaricus dataset:

In [ ]:
# read in data
dtrain = xgboost.DMatrix("data/agaricus.txt.train")

# specify parameters via map
param = {"max_depth": 2, "eta": 1, "objective": "binary:logistic"}
num_round = 2
bst = xgboost.train(param, dtrain, num_round)

In [ ]:
bentoml.xgboost.save_model("agaricus", bst)

In [ ]:
!bentoml models list

Test loading the model as a BentoML Runner instance:

In [ ]:
test_runner = bentoml.xgboost.get("agaricus:latest").to_runner()
test_runner.init_local()

# fmt: off
test_runner.run([[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
                  0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
                  1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
                  0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
                  0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
                  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
                  1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
                  0, 0, 0, 0, 0, 0, 1]])  # => array(0.01241208, dtype=float32)
# fmt: on

### Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

In [ ]:
%%writefile agaricus.py
import typing

import bentoml
import xgboost
from bentoml.io import NumpyNdarray, File

if typing.TYPE_CHECKING:
    import numpy as np

agaricus_runner = bentoml.xgboost.get("agaricus:latest").to_runner()

svc = bentoml.Service("agaricus", runners=[agaricus_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_data: "np.ndarray") -> "np.ndarray":
    return agaricus_runner.run(input_data)

Start a dev model server to test out the service defined above:

In [ ]:
!bentoml serve agaricus.py:svc

Open a new browser tab and test out the API endpoint from the web UI.

### Build a Bento for distribution and deployment

In [ ]:
!bentoml build

Starting a dev server with the Bento build:

In [ ]:
!bentoml serve agaricus:latest

## Hooray, your model is ready for production now 🎉

Bento is a standarized format for storing models alongside with all their API definitions, configuration, and environment settings. BentoML can start an REST API server serving a Bento, run a Bento as batch processing job on distributed dataset, or containerize all dependencies and models into a docker container image for easy production deployment.

### Optional: create a docker image for the model server

This will require docker to be installed and docker daemon to be running:

In [ ]:
!bentoml containerize agaricus:latest

## What's next?

* Learn more at http://docs.bentoml.org 📖
* Join [BentoML Slack community](https://join.slack.com/t/bentoml/shared_invite/enQtNjcyMTY3MjE4NTgzLTU3ZDc1MWM5MzQxMWQxMzJiNTc1MTJmMzYzMTYwMjQ0OGEwNDFmZDkzYWQxNzgxYWNhNjAxZjk4MzI4OGY1Yjg) 👈
* Follow us on [Twitter](https://twitter.com/bentomlai) 🐦
* Contribute to [BentoML on GitHub](https://github.com/bentoml/BentoML) 💻